In [15]:
import pandas as pd
from tqdm import tqdm
import numpy as np


In [2]:
train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [3]:
# train['customer_id'] = train['customer_id'].str[-16:].str.hex_to_int().astype('int64')
train['customer_id'] = train['customer_id'].str[-16:]
train['article_id'] = train.article_id.astype('int32')
train.t_dat = pd.to_datetime(train.t_dat)
train = train[['t_dat','customer_id','article_id']]
train.to_parquet('train.pqt',index=False)
print( train.shape )
train.head()

(31788324, 3)


,t_dat,customer_id,article_id
0,2018-09-20,ffe7ad4a1091e318,663713001
1,2018-09-20,ffe7ad4a1091e318,541518023
2,2018-09-20,8c5570910a014cc2,505221004
3,2018-09-20,8c5570910a014cc2,685687003
4,2018-09-20,8c5570910a014cc2,685687004


In [4]:
tmp = train.groupby('customer_id').t_dat.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.t_dat).dt.days
train = train.loc[train['diff_dat']<=6]
print('Train shape:',train.shape)

Train shape: (5181535, 5)


In [5]:
tmp = train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()
tmp.columns = ['customer_id','article_id','ct']
train = train.merge(tmp,on=['customer_id','article_id'],how='left')
train = train.sort_values(['ct','t_dat'],ascending=False)
train = train.drop_duplicates(['customer_id','article_id'])
train = train.sort_values(['ct','t_dat'],ascending=False)
train.head()

,t_dat,customer_id,article_id,max_dat,diff_dat,ct
1158548,2019-07-16,25df7287efeb5864,719348003,2019-07-16,0,100
80218,2018-10-04,3e3fc375f3dc0334,557247001,2018-10-04,0,86
2149868,2020-03-06,f369d633202f1980,852521001,2020-03-06,0,81
3423153,2020-07-06,31fb73d07178b8b6,685813001,2020-07-06,0,80
871769,2019-05-14,c02483cf582416e3,695545001,2019-05-14,0,80


In [18]:
df = train
vc = df.article_id.value_counts()
pairs = {}
for j,i in enumerate(tqdm(vc.index.values[:100])):
    #if j%10==0: print(j,', ',end='')
    USERS = df.loc[df.article_id==i.item(),'customer_id'].unique()
    vc2 = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i.item()),'article_id'].value_counts()
    pairs[i.item()] = vc2.index[0]

100%|██████████| 100/100 [00:23<00:00,  4.28it/s]


In [19]:
train['article_id2'] = train.article_id.map(pairs)

In [20]:
# RECOMMENDATION OF PAIRED ITEMS
train2 = train[['customer_id','article_id2']].copy()
train2 = train2.loc[train2.article_id2.notnull()]
train2 = train2.drop_duplicates(['customer_id','article_id2'])
train2 = train2.rename({'article_id2':'article_id'},axis=1)

In [21]:
train2.head()

,customer_id,article_id
1599585,f58ac00ca97d8e55,711053026.0
4865960,fb1430a6408e4dfa,850917003.0
1942946,8cd4b617da0b53f8,568597006.0
1412214,0874a8ca875ee9e1,568601006.0
4761058,5163ebf5a38dbd77,685814003.0


In [22]:
# CONCATENATE PAIRED ITEM RECOMMENDATION AFTER PREVIOUS PURCHASED RECOMMENDATIONS
train = train[['customer_id','article_id']]
train = pd.concat([train,train2],axis=0,ignore_index=True)
train.article_id = train.article_id.astype('int32')
train = train.drop_duplicates(['customer_id','article_id'])

In [23]:
# CONVERT RECOMMENDATIONS INTO SINGLE STRING
train.article_id = ' 0' + train.article_id.astype('str')
preds = pd.DataFrame( train.groupby('customer_id').article_id.sum().reset_index() )
preds.columns = ['customer_id','prediction']
preds.head()

,customer_id,prediction
0,000003dc9363af5d,0803757011 0715624010
1,000015cb3b12bea5,0732842001
2,000017199277bb1c,0780209001
3,00001a3c85978340,0925246001 0760084003 0760084006
4,00002554df8f5ce0,0876293001 0688537004 0684209004


In [26]:
train = pd.read_parquet('train.pqt')
train.t_dat = pd.to_datetime(train.t_dat)
train = train.loc[train.t_dat >= pd.to_datetime('2020-09-16')]
top12 = ' 0' + ' 0'.join(train.article_id.value_counts().index.astype('str')[:12])
print("Last week's top 12 popular items:")
print( top12 )

Last week's top 12 popular items:
 0924243001 0924243002 0918522001 0923758001 0866731001 0909370001 0751471001 0915529003 0915529005 0448509014 0762846027 0714790020


In [28]:
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
sub = sub[['customer_id']]
sub['customer_id_2'] = sub['customer_id'].str[-16:]
sub = sub.merge(preds.rename({'customer_id':'customer_id_2'},axis=1),\
    on='customer_id_2', how='left').fillna('')
del sub['customer_id_2']
sub.prediction = sub.prediction + top12
sub.prediction = sub.prediction.str.strip()
sub.prediction = sub.prediction.str[:131]
sub.to_csv(f'submission.csv',index=False)
sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0924243001 0924243002 0918522001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0924243001 0924243002 0918522001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0924243002 0918522001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001 0924243001 0924243002 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0927530004 0791587015 08...
